In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from clustering_utils import  build_cdlib_graph, build_ground_truth, get_ordered_communities, get_events_count
from utils import pre_processing_xes, discard_events_in_multiple_activities
import os
import pm4py
import networkx as nx

In [ ]:
# LOADING DATA
# event log not partitioned (single trace with a list of events)
event_log = xes_importer.apply('BP-Meets-IoT2020/d21p1/EventLogXESNoSegment.xes') # ('sim_22d1p_22/EventLogXESNoSegment.xes')
# event log partitioned (list of named traces)
event_log_labelled = xes_importer.apply('BP-Meets-IoT2020/d21p1/EventLogXES.xes') # ('sim_22d1p_22/EventLogXES.xes')

In [5]:
from utils import convert_to_dataframe, convert_to_event_log
from clustering_utils import get_events_count

In [6]:
event_log_df = convert_to_dataframe(event_log)

In [14]:
event_log_labelled_df = convert_to_dataframe(event_log_labelled)


In [7]:
from conversion import read_csv, format_dataframe, convert_to_event_log, write_xes

In [15]:
def split_at_3(df, output_file="abstraction/dataset-3am/split-3am", sort=False):
    """Convert a CSV file into a dataframe, segments it on a daily basis, sort it chronologically and save it as XES."""
    case_concept_name = 'case_id'

    if sort:
        df = df.sort_values(by=["time:timestamp", "eventId"])

    date_ref = None
    prev_date = None

    for i in df.index:
        df.at[i, 'reference:case:concept:name'] = df.at[i, 'case:concept:name']
        #case_id = case_concept_name
        if date_ref is None:
            date_ref = str(df.at[i, 'time:timestamp'])[0:10]
            case_id = date_ref

        # update case concept name if splitting is provided
        #hour = abstracted_csv.at[i, 'time:timestamp'].hour # str(abstracted_csv.at[i, 'time:timestamp'])[0:10]
        hour = int(str(df.at[i, 'time:timestamp'])[11:13])
        if hour > 3:
            current_date = str(df.at[i, 'time:timestamp'])[0:10]
            if current_date != date_ref:
                if prev_date is None:
                    prev_date = "-"
                print(
                    f"Split hour {hour} due to passed split at {3}. Current date: {df.at[i, 'time:timestamp']}, reference date {date_ref}, prev_date {prev_date}")
                date_ref = current_date
                case_id = date_ref
        prev_date = str(df.at[i, 'time:timestamp'])[0:15]
        df.at[i, 'case:concept:name'] = case_id

    df = format_dataframe(df, case_id='case:concept:name', activity_key='concept:name',
                                      timestamp_key='time:timestamp')


    abstracted = convert_to_event_log(df)
    write_xes(abstracted, output_file + "-splitted_at_3.xes")


In [16]:
split_at_3(event_log_labelled_df)

Split hour 18 due to passed split at 3. Current date: 2020-03-17 18:49:00+00:00, reference date 2020-03-16, prev_date 2020-03-16 22:3
Split hour 23 due to passed split at 3. Current date: 2020-03-16 23:34:00+00:00, reference date 2020-03-17, prev_date 2020-03-18 02:1
Split hour 4 due to passed split at 3. Current date: 2020-03-17 04:02:00+00:00, reference date 2020-03-16, prev_date 2020-03-17 00:0
Split hour 4 due to passed split at 3. Current date: 2020-03-18 04:17:00+00:00, reference date 2020-03-17, prev_date 2020-03-18 03:0
Split hour 4 due to passed split at 3. Current date: 2020-03-19 04:22:00+00:00, reference date 2020-03-18, prev_date 2020-03-18 21:1
Split hour 5 due to passed split at 3. Current date: 2020-03-20 05:39:00+00:00, reference date 2020-03-19, prev_date 2020-03-20 01:0
Split hour 4 due to passed split at 3. Current date: 2020-03-19 04:10:00+00:00, reference date 2020-03-20, prev_date 2020-03-20 05:4
Split hour 5 due to passed split at 3. Current date: 2020-03-20 05:

exporting log, completed traces ::   0%|          | 0/21 [00:00<?, ?it/s]